**Association Rule Mining**

The goal of this analysis is to find interesting relationships between products purchased in a store. By identifying strong associations between products, we can uncover patterns that can help make informed decisions about product promotions and recommendations.

In [14]:
# Import necessary libraries
import pandas as pd
# Load the dataset from github
url = 'https://raw.githubusercontent.com/karelsti/Data-Analyst-Portfolio/main/python_datamining/association_rule/store.csv'
df = pd.read_csv(url, encoding='latin1')
print(df.head())

   Row.ID         Order.ID  Order.Date   Ship.Date  Ship.Mode Customer.ID  \
0   42433     AG-2011-2040  01/01/2011  06/01/2011          4    TB-11280   
1   22253    IN-2011-47883  01/01/2011  08/01/2011          4    JH-15985   
2   48883     HU-2011-1220  01/01/2011  05/01/2011          3      AT-735   
3   11731  IT-2011-3647632  01/01/2011  05/01/2011          3    EM-14140   
4   22255    IN-2011-47883  01/01/2011  08/01/2011          4    JH-15985   

     Customer.Name  Segment         City            State  ... Category  \
0  Toby Braunhardt        1  Constantine      Constantine  ...        2   
1      Joseph Holt        1  Wagga Wagga  New South Wales  ...        2   
2    Annie Thurman        1     Budapest         Budapest  ...        2   
3     Eugene Moren        3    Stockholm        Stockholm  ...        2   
4      Joseph Holt        1  Wagga Wagga  New South Wales  ...        1   

   Sub.Category                 Product.Name    Sales Quantity  Discount  \
0       St

In [2]:
# Checking the structure of the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Row.ID          51290 non-null  int64  
 1   Order.ID        51290 non-null  object 
 2   Order.Date      51290 non-null  object 
 3   Ship.Date       51290 non-null  object 
 4   Ship.Mode       51290 non-null  int64  
 5   Customer.ID     51290 non-null  object 
 6   Customer.Name   51290 non-null  object 
 7   Segment         51290 non-null  int64  
 8   City            51290 non-null  object 
 9   State           51290 non-null  object 
 10  Country         51290 non-null  object 
 11  Postal.Code     9994 non-null   float64
 12  Market          51290 non-null  object 
 13  Region          51290 non-null  object 
 14  Product.ID      51290 non-null  object 
 15  Category        51290 non-null  int64  
 16  Sub.Category    51290 non-null  object 
 17  Product.Name    51290 non-null 

In [3]:
# Summary Statistics to help identify any outliers
df.describe()

,Row.ID,Ship.Mode,Segment,Postal.Code,Category,Sales,Quantity,Discount,Profit,Shipping.Cost,Status,Order.Priority
count,51290.00000,51290.000000,51290.000000,9994.000000,51290.000000,51290.000000,51290.000000,51290.000000,51290.000000,51290.000000,51290.0,51290.000000
mean,25645.50000,3.254709,1.665139,55190.379428,2.005167,246.490581,3.476545,0.142908,28.610983,26.375915,1.0,3.118308
std,14806.29199,1.083110,0.766199,32063.693350,0.624701,487.565362,2.278766,0.212280,174.340972,57.296804,0.0,1.081088
min,1.00000,1.000000,1.000000,1040.000000,1.000000,0.444000,1.000000,0.000000,-6599.978000,0.000000,1.0,1.000000
25%,12823.25000,3.000000,1.000000,23223.000000,2.000000,30.758625,2.000000,0.000000,0.000000,2.610000,1.0,2.000000
50%,25645.50000,4.000000,1.000000,56430.500000,2.000000,85.053000,3.000000,0.000000,9.240000,7.790000,1.0,4.000000
75%,38467.75000,4.000000,2.000000,90008.000000,2.000000,251.053200,5.000000,0.200000,36.810000,24.450000,1.0,4.000000
max,51290.00000,4.000000,3.000000,99301.000000,3.000000,22638.480000,14.000000,0.850000,8399.976000,933.570000,1.0,4.000000


In [4]:
(df['Profit'] < 0).sum()
# The result indicates that negative profit from an order occurs 12,544 times

12544

In [5]:
# Let's investigate if there are any missing values in each column
df.isna().sum()
# The 'Postal.Code' column has missing values, but since it's not relevant to our analysis, we'll ignore them

Row.ID                0
Order.ID              0
Order.Date            0
Ship.Date             0
Ship.Mode             0
Customer.ID           0
Customer.Name         0
Segment               0
City                  0
State                 0
Country               0
Postal.Code       41296
Market                0
Region                0
Product.ID            0
Category              0
Sub.Category          0
Product.Name          0
Sales                 0
Quantity              0
Discount              0
Profit                0
Shipping.Cost         0
Status                0
Order.Priority        0
dtype: int64

In [6]:
# I will now check the number of unuqie orders
unique_orders = df['Order.ID'].nunique()
print('Unique Orders:',unique_orders)
unique_products = df['Product.ID'].nunique()
print('Unique Products:',unique_products)
cnt_products = df['Product.ID'].count()
print(f'There are approximately',round(cnt_products/unique_orders),'products per one order')
# we can see that one order will have two items on average

Unique Orders: 25035
Unique Products: 10292
There are approximately 2 products per one order


In [7]:
df['Row.ID'].nunique()
# The number of unique RowIDs is equivalent to the total number of rows, hence we don't have any duplicated rows.

51290

**Most popular products**

Displaying most purchased products.

In [8]:
df['Product.Name'].value_counts().head(10)

Product.Name
Staples                                  227
Cardinal Index Tab, Clear                 92
Eldon File Cart, Single Width             90
Rogers File Cart, Single Width            84
Ibico Index Tab, Clear                    83
Sanford Pencil Sharpener, Water Color     80
Smead File Cart, Single Width             77
Stanley Pencil Sharpener, Water Color     75
Acco Index Tab, Clear                     75
Avery Index Tab, Clear                    74
Name: count, dtype: int64

**Frequent Customers**

Displaying customers with the most amount of orders.

In [9]:
top_customers = df.groupby(['Customer.ID', 'Customer.Name'])['Product.Name'].count()
top_customers.sort_values(ascending=False).head(10)

Customer.ID  Customer.Name      
PO-18850     Patrick O'Brill        97
BE-11335     Bill Eplett            94
JG-15805     John Grady             90
SW-20755     Steven Ward            89
EM-13960     Eric Murdock           85
MY-18295     Muhammed Yedwab        85
ZC-21910     Zuschuss Carroll       84
MP-17965     Michael Paige          84
CK-12205     Chloris Kastensmidt    83
AF-10870     Art Ferguson           81
Name: Product.Name, dtype: int64

**FP-growth vs Apriori Algorithm**

I have decided to use the FP growth algorithm because the Apriori algorithm works by iteratively generating candidate itemsets and counting their support which requires a lot of computational power and memory. FP-growth on the other hand uses FP-tree data structure and eliminates candidate generation and support counting.

In [10]:
pip install mlxtend
# Installing the mlxtend package

Note: you may need to restart the kernel to use updated packages.


**Transactional Data**

The following code converts categorical product names into binary columns (1 or 0), representing whether a product was part of an order or not. This format is required for association rule mining.

In [11]:
transaction_data_name = df[['Order.ID','Product.Name']]
transaction_data_name = pd.get_dummies(transaction_data_name['Product.Name']).groupby(transaction_data_name['Order.ID']).max()
transaction_data_name

,"""While you Were Out"" Message Book, One Form per Page","#10 Gummed Flap White Envelopes, 100/Box",#10 Self-Seal White Envelopes,"#10 White Business Envelopes,4 1/8 x 9 1/2","#10- 4 1/8"" x 9 1/2"" Recycled Envelopes","#10- 4 1/8"" x 9 1/2"" Security-Tint Envelopes","#10-4 1/8"" x 9 1/2"" Premium Diagonal Seam Envelopes",#6 3/4 Gummed Flap White Envelopes,"1.7 Cubic Foot Compact ""Cube"" Office Refrigerators","1/4 Fold Party Design Invitations & White Envelopes, 24 8-1/2"" X 11"" Cards, 25 Env./Pack",...,Zebra ZM400 Thermal Label Printer,Zebra Zazzle Fluorescent Highlighters,Zipper Ring Binder Pockets,i.Sound Portable Power - 8000 mAh,iHome FM Clock Radio with Lightning Dock,"iKross Bluetooth Portable Keyboard + Cell Phone Stand Holder + Brush for Apple iPhone 5S 5C 5, 4S 4",iOttie HLCRIO102 Car Mount,iOttie XL Car Mount,invisibleSHIELD by ZAGG Smudge-Free Screen Protector,netTALK DUO VoIP Telephone Service
Order.ID,,,,,,,,,,,,,,,,,,,,,
AE-2011-9160,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
AE-2013-1130,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
AE-2013-1530,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
AE-2014-2840,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
AE-2014-3830,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZI-2014-7160,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ZI-2014-7610,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ZI-2014-9540,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


**Frequent Itemsets**

The FP-Growth algorithm identifies frequent itemsets with a minimum support threshold of 0.0001.
Support measures how frequently items occur together in one order.

In [12]:
from mlxtend.frequent_patterns import fpgrowth
frequent_itemsets_name=fpgrowth(transaction_data_name, min_support=0.0001, use_colnames=True)
frequent_itemsets_name

,support,itemsets
0,0.001278,"(Fellowes File Cart, Industrial)"
1,0.000519,"(Epson Calculator, Red)"
2,0.001758,"(Bush Stackable Bookrack, Pine)"
3,0.000599,"(Accos Paper Clips, Bulk Pack)"
4,0.001198,"(Tenex Folders, Blue)"
...,...,...
3542,0.000120,"(Enermax Numeric Keypad, Bluetooth, SanDisk Me..."
3543,0.000120,"(Stockwell Thumb Tacks, Metal, Wilson Jones Bi..."
3544,0.000120,"(Boston Canvas, Easy-Erase, Office Star Rockin..."
3545,0.000120,"(Rogers File Cart, Single Width, Elite Ruler, ..."


**Resulting association rules**

The following association rules have a confidence greater than 0.5, meaning that if the antecedent is purchased, there is at least a 50% chance that the consequent will also be bought in the same order. The output displays the strongest association rules.

In [13]:
from mlxtend.frequent_patterns import association_rules
rules_name = association_rules(frequent_itemsets_name, metric="confidence", min_threshold=0.5)
rules_name

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Hon Olson Stacker Chairs),(Staples),0.00016,0.008868,0.00012,0.75,84.577703,0.000118,3.964530,0.988334
1,(Hoover Shoulder Vac Commercial Portable Vacuum),(Staples),0.00024,0.008868,0.00012,0.50,56.385135,0.000118,1.982265,0.982500
